In [3]:
# Your code here
#Librearias
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from imblearn.metrics import specificity_score
from pickle import dump

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
df = df.drop_duplicates().reset_index(drop = True)
df.shape

(768, 9)

In [7]:
#Haremos la seleccion de los datos de entenaminenot y test
#En este caso no es necesario normalizar las variables
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
60,2,84,0,0,0,0.0,0.304,21
618,9,112,82,24,0,28.2,1.282,50
346,1,139,46,19,83,28.7,0.654,22
294,0,161,50,0,0,21.9,0.254,65
231,6,134,80,37,370,46.2,0.238,46


In [8]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
668,6,98,58,33,190,34.0,0.430,43
324,2,112,75,32,0,35.7,0.148,21
624,2,108,64,0,0,30.8,0.158,21
690,8,107,80,0,0,24.6,0.856,34
473,7,136,90,0,0,29.9,0.210,50


In [9]:
#Lo convertiremos a CSV
'''
X_train_csv=X_train
X_test_csv=X_test
X_train_csv["Outcome"] = y_train.values
X_test_csv["Outcome"] = y_test.values
X_train_csv.to_csv("../data/processed/clean_train.csv", index = False)
X_test_csv.to_csv("../data/processed/clean_test.csv", index = False)
'''

'\nX_train_csv=X_train\nX_test_csv=X_test\nX_train_csv["Outcome"] = y_train.values\nX_test_csv["Outcome"] = y_test.values\nX_train_csv.to_csv("../data/processed/clean_train.csv", index = False)\nX_test_csv.to_csv("../data/processed/clean_test.csv", index = False)\n'

In [10]:
class CustomBoosting:

    def __init__(self, n_estimators=100, learning_rate=0.01, max_depth=10, min_samples_leaf=10, max_features=1.0, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.estimadores = []

    def muestreo(self, X, y, pesos):
        np.random.seed(self.random_state)
        idx_sample = np.random.choice(len(X), size=len(X), p=pesos)
        return X.iloc[idx_sample], y.iloc[idx_sample]

    def get_error(self, y, y_pred, pesos):
        instancias_erroneas = (y != y_pred)
        pesos_instancias_erroneas = pesos[instancias_erroneas]
        suma_pesos_erroneas = np.sum(pesos_instancias_erroneas)
        error_ponderado = suma_pesos_erroneas / np.sum(pesos)
        return instancias_erroneas, error_ponderado

    def get_estimator_importance(self, error_ponderado):
        ratio_acierto_error = (1 - error_ponderado) / error_ponderado
        ratio_log = np.log(ratio_acierto_error)
        estimator_imp = self.learning_rate * ratio_log
        return estimator_imp

    def update_weights(self, estimator_imp, instancias_erroneas, pesos):
        actualizacion_pesos = np.where(instancias_erroneas, np.exp(estimator_imp), np.exp(-estimator_imp))
        pesos = pesos * actualizacion_pesos
        pesos = pesos / np.sum(pesos)
        return pesos

    def fit(self, X, y):
        pesos = np.ones(len(X)) / len(X)
        for _ in range(self.n_estimators):
            X_sampled, y_sampled = self.muestreo(X, y, pesos)
            estimador = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X_sampled, y_sampled)
            y_pred = estimador.predict(X)
            instancias_erroneas, error_ponderado = self.get_error(y, y_pred, pesos)
            estimator_imp = self.get_estimator_importance(error_ponderado)
            self.estimadores.append((estimator_imp, estimador))
            pesos = self.update_weights(estimator_imp, instancias_erroneas, pesos)

    def predict(self, X):
        final_predictions = np.zeros(len(X))
        for estimator_imp, estimator in self.estimadores:
            predictions = estimator.predict(X)
            preds_ponderadas = predictions * estimator_imp
            final_predictions += preds_ponderadas
        mean_val = np.mean(final_predictions)
        y_pred = np.where(final_predictions < mean_val, 0, 1)
        return y_pred


In [11]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [12]:
# Entrenamos el modelo de Boosting
model = CustomBoosting(n_estimators=10, learning_rate=0.01, max_depth=7, min_samples_leaf=7, max_features=X_train.shape[1]//2)
model.fit(X_train, y_train)

In [13]:
# Realizamos predicciones en el conjunto de entrenamiento y prueba
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [14]:
# Evaluamos el rendimiento del modelo
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.840391,0.787879,0.843689,0.730924,0.854460,0.832918
Test,0.779221,0.716667,0.779798,0.661538,0.781818,0.777778
Diferencia,0.061170,0.071212,0.063891,0.069385,0.072642,0.055140


In [15]:
#Analizacermos las predicciones
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1])

In [16]:
accuracy_score(y_test, y_pred)

0.7792207792207793